quick test

In [5]:
import numpy as np
import scipy as sp
import pandas as pd

In [158]:
def f_objective(theta, X, y, l2_param=1):
    '''
    Args:
        theta: 1D numpy array of size num_features
        X: 2D numpy array of size (num_instances, num_features)
        y: 1D numpy array of size num_instances
        l2_param: regularization parameter

    Returns:
        objective: scalar value of objective function
    '''
    ## break this in to two parts regulization and loss term 
    print(X.shape)
    print(y.shape)
    print(theta.shape)
    regularization_term=l2_param*np.dot(theta,theta) ## l_2 regulairzation term 
    print(regularization_term.shape)
    # loss_term=np.logaddexp(np.zeros(X.shape[0]), X@theta*y) ## loss term. 
    # return regularization_term+loss_term
def fit_logistic_reg(X, y, objective_function, l2_param=1):
    '''
    Args:
        X: 2D numpy array of size (num_instances, num_features)
        y: 1D numpy array of size num_instances
        objective_function: function returning the value of the objective
        l2_param: regularization parameter
    Returns:
        optimal_theta: 1D numpy array of size num_features
    '''
    x0=np.ones(X.shape[1])
    res=sp.optimize.minimize(objective_function, x0, args=( X, y ), method="Nelder-Mead")
    print(res)
    return res.x
        

## load and pre-process data 

In [142]:
X_train= pd.read_csv(r'/home/buzgalbraith/school/spring_2023/Machine_Learning_spring_2023/homework_code/hw4/logistic-code/X_train.txt', sep=",")
y_train=pd.read_csv(r'/home/buzgalbraith/school/spring_2023/Machine_Learning_spring_2023/homework_code/hw4/logistic-code/y_train.txt')
X_test=pd.read_csv(r'/home/buzgalbraith/school/spring_2023/Machine_Learning_spring_2023/homework_code/hw4/logistic-code/X_val.txt')
y_test=pd.read_csv(r'/home/buzgalbraith/school/spring_2023/Machine_Learning_spring_2023/homework_code/hw4/logistic-code/y_val.txt')

def standardize(X_train,X_test):
    X_train= (X_train - np.mean(X_train) ) / np.std(X_train)
    X_test=( X_test - np.mean(X_test) ) / np.std(X_test)
    return X_train, X_test
def add_bias_term(X_train, X_test):
    X_train["bias"]=np.ones(X_train.shape[0])
    X_test["bias"]=np.ones(X_test.shape[0])
    return X_train, X_test
X_train, X_test=standardize(X_train,X_test)
X_train, X_test=add_bias_term(X_train, X_test)

/home/buzgalbraith/miniconda3/envs/machine_learning/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3430: FutureWarning: In a future version, DataFrame.mean(axis=None) will return a scalar mean over the entire DataFrame. To retain the old behavior, use 'frame.mean(axis=0)' or just 'frame.mean()'
  return mean(axis=axis, dtype=dtype, out=out, **kwargs)


In [159]:
w=fit_logistic_reg( X=X_train, y=y_train,objective_function=f_objective )
w


(1599, 21)
(1599, 1)
(21,)
()
(1599, 21)
(1599, 1)
(21,)
()
(1599, 21)
(1599, 1)
(21,)
()
(1599, 21)
(1599, 1)
(21,)
()
(1599, 21)
(1599, 1)
(21,)
()
(1599, 21)
(1599, 1)
(21,)
()
(1599, 21)
(1599, 1)
(21,)
()
(1599, 21)
(1599, 1)
(21,)
()
(1599, 21)
(1599, 1)
(21,)
()
(1599, 21)
(1599, 1)
(21,)
()
(1599, 21)
(1599, 1)
(21,)
()
(1599, 21)
(1599, 1)
(21,)
()
(1599, 21)
(1599, 1)
(21,)
()
(1599, 21)
(1599, 1)
(21,)
()
(1599, 21)
(1599, 1)
(21,)
()
(1599, 21)
(1599, 1)
(21,)
()
(1599, 21)
(1599, 1)
(21,)
()
(1599, 21)
(1599, 1)
(21,)
()
(1599, 21)
(1599, 1)
(21,)
()
(1599, 21)
(1599, 1)
(21,)
()
(1599, 21)
(1599, 1)
(21,)
()
(1599, 21)
(1599, 1)
(21,)
()
(1599, 21)
(1599, 1)
(21,)
()
(1599, 21)
(1599, 1)
(21,)
()
(1599, 21)
(1599, 1)
(21,)
()
(1599, 21)
(1599, 1)
(21,)
()
(1599, 21)
(1599, 1)
(21,)
()
(1599, 21)
(1599, 1)
(21,)
()
(1599, 21)
(1599, 1)
(21,)
()
(1599, 21)
(1599, 1)
(21,)
()
(1599, 21)
(1599, 1)
(21,)
()
(1599, 21)
(1599, 1)
(21,)
()
(1599, 21)
(1599, 1)
(21,)
()
(1599, 21)

array([ 0.02282966,  0.01089919, -0.0063951 , -0.04045788, -0.01126671,
        0.01028561, -0.00330994, -0.05390559,  0.0002492 , -0.02823302,
       -0.02867952,  0.04229097,  0.00788701, -0.01383145, -0.00466752,
       -0.00555376, -0.01070479, -0.01349332,  0.00952429, -0.03180587,
        0.05454126])

/home/buzgalbraith/miniconda3/envs/machine_learning/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3430: FutureWarning: In a future version, DataFrame.mean(axis=None) will return a scalar mean over the entire DataFrame. To retain the old behavior, use 'frame.mean(axis=0)' or just 'frame.mean()'
  return mean(axis=axis, dtype=dtype, out=out, **kwargs)


,-3.658641073349477857e-01,5.246133451644787726e-01,-1.486924449634822620e+00,-9.091259868125525223e-01,-2.383979479960978587e+00,2.021899537027167071e+00,-7.527635042691687151e-01,8.124049171730468588e-01,3.368428287502183149e-01,-1.236076047206864092e-01,6.110367201864366216e-01,-1.159904898509284132e+00,7.765556748173368939e-01,-3.751333038994605351e-01,3.674861257782893742e-01,-3.694675621239626451e-01,3.743877480297692073e-01,4.316403285195357653e-01,-1.241969650011315540e+00,-2.946980839896981563e-01
0,0.171943,1.016441,1.445831,2.323920,1.018202,1.228993,0.980445,0.934771,1.032695,4.100908,0.913525,0.996323,1.216734,1.007799,0.994330,0.949872,1.022402,0.996716,0.937537,0.902711
1,-0.732008,0.829349,1.011722,0.946986,0.966247,0.929585,1.007066,0.980351,0.975527,0.979663,0.981363,1.002765,1.003979,1.015780,1.015344,0.970027,0.989774,1.000637,0.963935,0.945778
2,-0.063609,1.058113,1.006136,0.967052,0.973979,0.988599,0.989061,1.214515,0.938367,0.988453,1.046677,0.997246,1.002123,0.965093,0.991017,0.970892,-14.772895,1.004142,0.893539,1.067529
3,-0.428381,3.678216,1.005155,1.009694,1.047526,0.974991,0.974034,1.132324,1.014493,1.013034,1.009665,0.977510,1.002952,1.042468,1.010819,1.069772,1.124090,0.991910,1.533270,1.034387
4,0.202413,0.972550,1.013451,0.979894,0.974353,1.066375,0.988333,1.023617,0.989813,0.988350,1.238480,0.986287,1.003675,1.029682,0.985463,1.248942,1.257574,1.000835,0.580554,-0.817508
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1594,0.154326,0.383549,1.004550,0.959186,0.664693,0.987770,0.977398,-0.059318,0.991903,0.989537,0.984867,1.091914,1.001642,1.037115,1.002157,0.850993,1.016282,0.999712,0.971731,1.105680
1595,-1.152557,0.644293,1.014275,0.977398,1.023379,1.019181,1.030232,0.908549,-1.682266,0.987097,0.981425,1.036029,1.003965,0.988904,0.981357,0.966734,1.017578,1.004915,1.040414,1.017864
1596,-0.001771,1.077331,1.197650,0.930842,0.971910,0.992183,1.326616,0.954742,0.975785,0.950345,1.367023,1.003207,1.027649,0.946764,1.005920,1.122040,1.006687,0.992684,0.910024,0.928338
1597,0.164620,1.026105,1.035015,0.980013,0.966469,0.962453,0.690752,0.992967,0.993067,0.986282,1.324810,1.010486,1.006501,1.042257,0.993592,1.046213,1.007983,0.997846,1.084207,0.975411
